In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%pip install patool
import patoolib
patoolib.extract_archive("/content/drive/My Drive/Adience/alignedGender.rar", outdir="/content")

     |████████████████████████████████| 81kB 2.6MB/s 
patool: Extracting /content/drive/My Drive/Adience/alignedGender.rar ...
patool: running /usr/bin/unrar x -- "/content/drive/My Drive/Adience/alignedGender.rar"
patool:     with cwd='/content'
patool: ... /content/drive/My Drive/Adience/alignedGender.rar extracted to `/content'.


'/content'

In [ ]:
import keras
import  tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, PReLU, ReLU, Input, GlobalAveragePooling2D
from keras import optimizers, regularizers
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
np.random.seed(48)
from keras.applications import vgg19
from keras.models import Model, load_model
from keras.optimizers import Adam
# input_shape = (224, 224, 3)
# vgg = vgg19.VGG19(include_top=False ,weights='imagenet', 
#                                      input_shape=input_shape)   
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.


In [ ]:
seed = 1234
rn.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

### Group Normalization

In [ ]:
from keras.engine import Layer, InputSpec
from keras import initializers
from keras import regularizers
from keras import constraints
from keras import backend as K

from keras.utils.generic_utils import get_custom_objects


class GroupNormalization(Layer):
    """Group normalization layer
    Group Normalization divides the channels into groups and computes within each group
    the mean and variance for normalization. GN's computation is independent of batch sizes,
    and its accuracy is stable in a wide range of batch sizes
    # Arguments
        groups: Integer, the number of groups for Group Normalization.
        axis: Integer, the axis that should be normalized
            (typically the features axis).
            For instance, after a `Conv2D` layer with
            `data_format="channels_first"`,
            set `axis=1` in `BatchNormalization`.
        epsilon: Small float added to variance to avoid dividing by zero.
        center: If True, add offset of `beta` to normalized tensor.
            If False, `beta` is ignored.
        scale: If True, multiply by `gamma`.
            If False, `gamma` is not used.
            When the next layer is linear (also e.g. `nn.relu`),
            this can be disabled since the scaling
            will be done by the next layer.
        beta_initializer: Initializer for the beta weight.
        gamma_initializer: Initializer for the gamma weight.
        beta_regularizer: Optional regularizer for the beta weight.
        gamma_regularizer: Optional regularizer for the gamma weight.
        beta_constraint: Optional constraint for the beta weight.
        gamma_constraint: Optional constraint for the gamma weight.
    # Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
    # Output shape
        Same shape as input.
    # References
        - [Group Normalization](https://arxiv.org/abs/1803.08494)
    """

    def __init__(self,
                 groups=32,
                 axis=-1,
                 epsilon=1e-5,
                 center=True,
                 scale=True,
                 beta_initializer='zeros',
                 gamma_initializer='ones',
                 beta_regularizer=None,
                 gamma_regularizer=None,
                 beta_constraint=None,
                 gamma_constraint=None,
                 **kwargs):
        super(GroupNormalization, self).__init__(**kwargs)
        self.supports_masking = True
        self.groups = groups
        self.axis = axis
        self.epsilon = epsilon
        self.center = center
        self.scale = scale
        self.beta_initializer = initializers.get(beta_initializer)
        self.gamma_initializer = initializers.get(gamma_initializer)
        self.beta_regularizer = regularizers.get(beta_regularizer)
        self.gamma_regularizer = regularizers.get(gamma_regularizer)
        self.beta_constraint = constraints.get(beta_constraint)
        self.gamma_constraint = constraints.get(gamma_constraint)

    def build(self, input_shape):
        dim = input_shape[self.axis]

        if dim is None:
            raise ValueError('Axis ' + str(self.axis) + ' of '
                             'input tensor should have a defined dimension '
                             'but the layer received an input with shape ' +
                             str(input_shape) + '.')

        if dim < self.groups:
            raise ValueError('Number of groups (' + str(self.groups) + ') cannot be '
                             'more than the number of channels (' +
                             str(dim) + ').')

        if dim % self.groups != 0:
            raise ValueError('Number of groups (' + str(self.groups) + ') must be a '
                             'multiple of the number of channels (' +
                             str(dim) + ').')

        self.input_spec = InputSpec(ndim=len(input_shape),
                                    axes={self.axis: dim})
        shape = (dim,)

        if self.scale:
            self.gamma = self.add_weight(shape=shape,
                                         name='gamma',
                                         initializer=self.gamma_initializer,
                                         regularizer=self.gamma_regularizer,
                                         constraint=self.gamma_constraint)
        else:
            self.gamma = None
        if self.center:
            self.beta = self.add_weight(shape=shape,
                                        name='beta',
                                        initializer=self.beta_initializer,
                                        regularizer=self.beta_regularizer,
                                        constraint=self.beta_constraint)
        else:
            self.beta = None
        self.built = True

    def call(self, inputs, **kwargs):
        input_shape = K.int_shape(inputs)
        tensor_input_shape = K.shape(inputs)

        # Prepare broadcasting shape.
        reduction_axes = list(range(len(input_shape)))
        del reduction_axes[self.axis]
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis] // self.groups
        broadcast_shape.insert(1, self.groups)

        reshape_group_shape = K.shape(inputs)
        group_axes = [reshape_group_shape[i] for i in range(len(input_shape))]
        group_axes[self.axis] = input_shape[self.axis] // self.groups
        group_axes.insert(1, self.groups)

        # reshape inputs to new group shape
        group_shape = [group_axes[0], self.groups] + group_axes[2:]
        group_shape = K.stack(group_shape)
        inputs = K.reshape(inputs, group_shape)

        group_reduction_axes = list(range(len(group_axes)))
        group_reduction_axes = group_reduction_axes[2:]

        mean = K.mean(inputs, axis=group_reduction_axes, keepdims=True)
        variance = K.var(inputs, axis=group_reduction_axes, keepdims=True)

        inputs = (inputs - mean) / (K.sqrt(variance + self.epsilon))

        # prepare broadcast shape
        inputs = K.reshape(inputs, group_shape)
        outputs = inputs

        # In this case we must explicitly broadcast all parameters.
        if self.scale:
            broadcast_gamma = K.reshape(self.gamma, broadcast_shape)
            outputs = outputs * broadcast_gamma

        if self.center:
            broadcast_beta = K.reshape(self.beta, broadcast_shape)
            outputs = outputs + broadcast_beta

        outputs = K.reshape(outputs, tensor_input_shape)

        return outputs

    def get_config(self):
        config = {
            'groups': self.groups,
            'axis': self.axis,
            'epsilon': self.epsilon,
            'center': self.center,
            'scale': self.scale,
            'beta_initializer': initializers.serialize(self.beta_initializer),
            'gamma_initializer': initializers.serialize(self.gamma_initializer),
            'beta_regularizer': regularizers.serialize(self.beta_regularizer),
            'gamma_regularizer': regularizers.serialize(self.gamma_regularizer),
            'beta_constraint': constraints.serialize(self.beta_constraint),
            'gamma_constraint': constraints.serialize(self.gamma_constraint)
        }
        base_config = super(GroupNormalization, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape


get_custom_objects().update({'GroupNormalization': GroupNormalization})


if __name__ == '__main__':
    from keras.layers import Input
    from keras.models import Model
    ip = Input(shape=(None, None, 4))
    #ip = Input(batch_shape=(100, None, None, 2))
    x = GroupNormalization(groups=2, axis=-1, epsilon=0.1)(ip)
    model = Model(ip, x)
    model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 4)     0         
_________________________________________________________________
group_normalization_1 (Group (None, None, None, 4)     8         
Total params: 8
Trainable params: 8
Non-trainable params: 0
_________________________________________________________________


### Efficient Net


In [ ]:
!pip install efficientnet
import efficientnet.keras as efn 

In [ ]:
class FixedImageDataGenerator(ImageDataGenerator):
  def standardize(self, x):
    if self.featurewise_center:
      img = tf.keras.applications.resnet50.preprocess_input(x)
      # x = ((x/255.) - 0.5) * 2.
    return x

In [ ]:
IMG_WIDTH = 227
IMG_HEIGHT = 227
CHANNELS = 3

In [ ]:
data_gen_args = dict(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

train_datagen = FixedImageDataGenerator(**data_gen_args)
train_generator = train_datagen.flow_from_directory(
        '/content/alignedGender/train',
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=32,
        class_mode='categorical')

data_gen_args = dict(
    featurewise_center=True,
    featurewise_std_normalization=True,
    # rotation_range=20,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # horizontal_flip=True,
)
validate_datagen = FixedImageDataGenerator(**data_gen_args)
validate_generator = validate_datagen.flow_from_directory(
        '/content/alignedGender/test',
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=32,
        class_mode='categorical')

Found 14047 images belonging to 2 classes.
Found 3445 images belonging to 2 classes.


In [ ]:
def efficientnetB0():
    model_input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, CHANNELS))
    baseModel = efn.EfficientNetB0(weights='imagenet', include_top=False, input_tensor=model_input)
    # for layer in baseModel.layers:
	#     layer.trainable = False
    for i, layer in enumerate(baseModel.layers):
        if "batch_normalization" in layer.name:
            baseModel.layers[i] = GroupNormalization(groups=32, axis=-1, epsilon=0.00001)
    X = baseModel.output
    X = GlobalAveragePooling2D()(X)
    X = Dense(512, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.1, l2=0.1))(X)
    X = Dropout(0.5)(X)
    X = Dense(2, activation="softmax")(X)
    model = Model(inputs=model_input, outputs=X)
    model.compile(optimizer=Adam(lr=1e-3, beta_1=0.9), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
efficientnetb0_model = efficientnetB0()

16809984/16804768 [==============================] - 1s 0us/step


In [ ]:
filepath= "effinet-gender-b2-v4.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_accuracy", mode="max", patience=5)
rlr = ReduceLROnPlateau(monitor='val_loss', 
                        factor=0.5, 
                        patience=2, 
                        verbose=1, 
                        mode='auto', 
                        min_delta=0.0001)
callbacks_list = [checkpoint, early, rlr]

In [ ]:
batch_size = 32
epochs = 10
history = efficientnetb0_model.fit_generator(train_generator, 
                              steps_per_epoch = train_generator.samples // batch_size, 
                              epochs = epochs,
                              validation_data = validate_generator,
                              validation_steps= validate_generator.samples // batch_size, 
                              callbacks = callbacks_list,
                              verbose=1)

Epoch 1/10
438/438 [==============================] - 289s 660ms/step - loss: 0.7423 - accuracy: 0.9924 - val_loss: 0.8330 - val_accuracy: 0.9270

Epoch 00001: val_accuracy did not improve from 0.93818
Epoch 2/10
438/438 [==============================] - 291s 665ms/step - loss: 0.7467 - accuracy: 0.9935 - val_loss: 1.1699 - val_accuracy: 0.9241

Epoch 00002: val_accuracy did not improve from 0.93818
Epoch 3/10
438/438 [==============================] - 291s 664ms/step - loss: 0.7528 - accuracy: 0.9923 - val_loss: 0.8387 - val_accuracy: 0.9306

Epoch 00003: val_accuracy did not improve from 0.93818

Epoch 00003: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
Epoch 4/10
438/438 [==============================] - 290s 663ms/step - loss: 0.4745 - accuracy: 0.9935 - val_loss: 0.6917 - val_accuracy: 0.9317

Epoch 00004: val_accuracy did not improve from 0.93818
Epoch 5/10
438/438 [==============================] - 290s 662ms/step - loss: 0.4758 - accuracy: 0.9941 - val_l

In [ ]:
!cp  '/content/effinet-gender-b2-v4-938.hdf5' '/content/drive/My Drive/Adience'